In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from glob import glob
import itertools
from pao_file_utils import parse_pao_file
from sympy.physics.quantum.cg import CG

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
#https://github.com/cp2k/cp2k/blob/master/src/common/spherical_harmonics.F
def Y_l(r, l):
    """Real Spherical Harmonics"""
    assert r.shape[-1] == 3

    if l < 0:
        raise Exceptoin("Negative l value")
    elif l == 0:
        return np.sqrt(1.0 / (4.0 * np.pi))
    elif l == 1:
        pf = np.sqrt(3.0 / (4.0 * np.pi))
        return pf * r
    elif l == 2:
        x = r[..., 0]
        y = r[..., 1]
        z = r[..., 2]
        result = np.zeros(5)
        # m = 2
        pf = np.sqrt(15.0 / (16.0 * np.pi))
        result[0] = pf * x**2 - y**2
        # m = 1
        pf = np.sqrt(15.0 / (4.0 * np.pi))
        result[1] = pf * z * x
        # m = 0
        pf = np.sqrt(5.0 / (16.0 * np.pi))
        result[2] = pf * (3.0 * z**2 - 1.0)
        # m = -1
        pf = np.sqrt(15.0 / (4.0 * np.pi))
        result[3] = pf * z * y
        # m = -2
        pf = np.sqrt(15.0 / (16.0 * np.pi))
        result[4] = pf * 2.0 * x * y
    return result

In [4]:
def convolute(coords, atom2kind, central_atom, max_l):
    natoms = coords.shape[0]
    assert coords.shape[1] == 3
    kind_names = list(sorted(set(atom2kind)))
    integrals = []
    for l in range(max_l + 1):
        for sigma in [0.5, 1.0, 2.0, 3.0, 4.0]:
            for ikind in kind_names:
                integrals.append(np.zeros(2*l + 1))
                for iatom in range(natoms):
                    if atom2kind[iatom] == ikind and iatom != central_atom:
                        r = coords[central_atom] - coords[iatom]
                        angular_part = Y_l(r, l)
                        radial_part = np.exp(- np.dot(r,r) / sigma**2)
                        integrals[-1] += radial_part * angular_part
    return integrals

In [5]:
cg_cache = dict()

#TODO maybe rename lo -> lk ?
def get_clebsch_gordan_coefficients(li, lj, lo):
    global cg_cache
    key = (li, lj, lo)
    if key not in cg_cache:
        assert abs(li-lj) <= lo <= abs(li+lj)
        coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
        for mi in range(-li, li+1):
            for mj in range(-lj, lj+1):
                for mo in range(-lo, lo+1):
                    # https://docs.sympy.org/latest/modules/physics/quantum/cg.html
                    cg = CG(li, mi, lj, mj, lo, mo).doit()
                    coeffs[mi+li, mj+lj, mo+lo] = cg
        cg_cache[key] = coeffs
    return cg_cache[key]

In [6]:
def combinations(channels, max_l):
    """ returns all possbile combinations of input channels up to given max_l """
    output_channels = list()
    for channel_i, channel_j in itertools.combinations_with_replacement(channels, 2):
        assert len(channel_i.shape) == len(channel_j.shape) == 1
        li = (channel_i.size - 1) // 2
        lj = (channel_j.size - 1) // 2
        # There li + lj possible ways to combine the two channels.
        # We do all of them up to a max_l.
        lo_min = abs(li-lj)
        lo_max = min(li+lj, max_l)
        for lo in range(lo_min, lo_max + 1): # l of output
            channel_o = np.zeros(2*lo+1)
            cg = get_clebsch_gordan_coefficients(li, lj, lo)
            channel_o = np.einsum("i,j,ijo->o", channel_i, channel_j, cg)
            output_channels.append(channel_o)
    return output_channels

In [7]:
# Load training data and hard code metadata.
#pao_files = sorted(glob("2H2O_MD/frame_*/2H2O_pao44-1_0.pao"))
pao_files = sorted(glob("2H2O_rotations/*/2H2O_pao44-1_0.pao"))

prim_basis_shells = {
    'H': [2, 1, 0], # two s-shells, one p-shell, no d-shells
    'O': [2, 2, 1], # two s-shells, two p-shells, one d-shell
}

pao_basis_size = 4

In [8]:
class Sample:
    def __init__(self, channels, xblock):
        self.channels = channels
        self.xblock = xblock
        #TODO generalize
        self.s_channel_list = [c for c in channels if c.size == 1]
        self.p_channel_list = [c for c in channels if c.size == 3]
        self.d_channel_list = [c for c in channels if c.size == 5]

        self.s_channels = np.concatenate(self.s_channel_list)
        self.p_channels = np.stack(self.p_channel_list).T
        if (self.d_channel_list):
            self.d_channels = np.stack(self.d_channel_list).T
        else:
            self.d_channels = np.zeros(shape=[])

def build_dataset(kind_name, max_l):
    samples = []
    for fn in pao_files:
        kinds, atom2kind, coords, xblocks = parse_pao_file(fn)
        #kind_onehot = encode_kind(atom2kind)
        natoms = coords.shape[0]
        for iatom in range(natoms):
            if atom2kind[iatom] == kind_name:
                initial_channels = convolute(coords, atom2kind, iatom, max_l)
                comb_channels = combinations(initial_channels, max_l)
                sample = Sample(comb_channels, xblocks[iatom])
                samples.append(sample)

    print("\nDataset for: ", kind_name)
    print("samples: ", len(samples))
    print("s channels: ", len(samples[0].s_channel_list))
    print("p channels: ", len(samples[0].p_channel_list))
    print("d channels: ", len(samples[0].d_channel_list))
    return samples
    
def samples2xy(samples):
    s_input = np.array([s.s_channels for s in samples], np.float32)
    p_input = np.array([s.p_channels for s in samples], np.float32)
    d_input = np.array([s.d_channels for s in samples], np.float32)
    x_data = [s_input, p_input, d_input]
    y_data = np.array([s.xblock for s in samples], np.float32)
    return x_data, y_data

H_dataset = build_dataset("H", max_l=1)
#O_dataset = build_dataset("O", max_l=2)


Dataset for:  H
samples:  224
s channels:  110
p channels:  155
d channels:  0


In [9]:
def contract(args):
    assert len(args) == 2
    coeffs = args[0]
    channels = args[1]
    return tf.reduce_sum(coeffs[...,None,:] * channels, axis=-1)

#TODO implement custom layer instead of Dense + Lambda:

def build_model(first_sample, pao_basis_size, prim_basis_shells, num_hidden_layers):
    
    prim_basis_size = sum([(2*l+1)*n for l, n in enumerate(prim_basis_shells)])
    num_p_channels = len(first_sample.p_channel_list)
    num_d_channels = len(first_sample.d_channel_list)
    
    # define two sets of inputs
    s_input = layers.Input(shape=first_sample.s_channels.shape, name="s_input")
    p_input = layers.Input(shape=first_sample.p_channels.shape, name="p_input")
    d_input = layers.Input(shape=first_sample.d_channels.shape, name="d_input")
    
    #TODO: add more hidden layers
    hidden_out = s_input
    for i in range(num_hidden_layers):
        hidden_out = layers.Dense(8, activation='softmax', name=f"hidden_layer_{i}")(hidden_out)
            
    pao_output = []
    for i in range(pao_basis_size):
        for l, n in enumerate(prim_basis_shells):
            for j in range(n):
                name = f"{i}{l}{j}"
                if l == 0:
                    s_out = layers.Dense(1, name="coeffs_"+name)(hidden_out)
                    pao_output.append(s_out)
                elif l == 1:
                    p_coeffs = layers.Dense(num_p_channels, name="coeffs_"+name)(hidden_out)
                    contract_layer = layers.Lambda(contract, name="contr_"+name)
                    p_out = contract_layer([p_coeffs, p_input])
                    pao_output.append(p_out)
                elif l == 2:
                    d_coeffs = layers.Dense(num_d_channels, name="coeffs_"+name)(hidden_out)
                    contract_layer = layers.Lambda(contract, name="contr_"+name)
                    d_out = contract_layer([d_coeffs, d_input])
                    pao_output.append(d_out)
                else:
                    raise Exception("Not implemented")
    
    xvec = layers.concatenate(pao_output)
    xblock = layers.Reshape((pao_basis_size, prim_basis_size))(xvec) #TODO: maybe transpose?

    inputs = [s_input, p_input, d_input]
    model = keras.Model(inputs=inputs, outputs=xblock)
    #model.summary()
    return(model)

H_model = build_model(H_dataset[0], pao_basis_size, prim_basis_shells['H'], num_hidden_layers=2)
#O_model = build_model(O_dataset[0], pao_basis_size, prim_basis_shells['O'])

In [10]:
def loss_function(xblock_true, xblock_pred):
    #TODO: This might not be ideal as it implicitly forces the predicted basis vectors to be orthonormal.
    projector = tf.matmul(xblock_pred, xblock_pred, transpose_a=True)
    residual = xblock_true - tf.matmul(xblock_true, projector)
    return tf.reduce_mean(tf.pow(residual, 2))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
H_model.compile(optimizer, loss=loss_function)

In [15]:
class TrainingLogger(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch_counter = 0
    def on_epoch_end(self, epoch, logs={}):
        self.epoch_counter += 1
        if self.epoch_counter % 10 == 0:
            print("Epoch: {} Loss: {}".format(self.epoch_counter, logs['loss']))

x_train, y_train = samples2xy(H_dataset[:1])
x_test, y_test = samples2xy(H_dataset[1:])
H_model.fit(x_train, y_train, epochs=300, verbose=0, callbacks=[TrainingLogger()])
loss = H_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", loss)

Epoch: 10 Loss: 1.2102818747194988e-13
Epoch: 20 Loss: 6.909750464807523e-15
Epoch: 30 Loss: 3.7952280207420586e-14
Epoch: 40 Loss: 5.316649965875393e-14
Epoch: 50 Loss: 3.358841050887969e-14
Epoch: 60 Loss: 2.2103151963879727e-14
Epoch: 70 Loss: 4.371433702759879e-14
Epoch: 80 Loss: 6.376704584287393e-14
Epoch: 90 Loss: 3.4806186389015406e-14
Epoch: 100 Loss: 1.464162179085765e-13
Epoch: 110 Loss: 2.588901315547787e-14
Epoch: 120 Loss: 2.0942276369008764e-14
Epoch: 130 Loss: 1.8161860226460422e-14
Epoch: 140 Loss: 5.5202370452533955e-14
Epoch: 150 Loss: 2.590011606335048e-14
Epoch: 160 Loss: 8.388428840433448e-14
Epoch: 170 Loss: 1.5729847708655792e-13
Epoch: 180 Loss: 1.7152113605291286e-13
Epoch: 190 Loss: 1.3816100769958178e-13
Epoch: 200 Loss: 1.967224939804918e-13
Epoch: 210 Loss: 2.6918744333191344e-14
Epoch: 220 Loss: 6.044470479693587e-14
Epoch: 230 Loss: 6.043637676899846e-14
Epoch: 240 Loss: 2.6578600973749256e-13
Epoch: 250 Loss: 9.673511991437067e-14
Epoch: 260 Loss: 2.982